In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import sys

In [2]:
classes_path = "../data/eval_classes.csv"
anomalies_path = "../data/eval_anomalies.csv"

cls = pd.read_csv(classes_path, delimiter=',', quoting=csv.QUOTE_MINIMAL, error_bad_lines=True,
                  header=None, names=["class_name", "score"])
ans = pd.read_csv(anomalies_path, delimiter=',', quoting=csv.QUOTE_MINIMAL, error_bad_lines=True,
                  header=None, names=["file_name", "class_name", "method", "score"])

In [3]:
print(cls.shape)
cls.loc[:, "score"].value_counts()

(43, 2)


4    11
2    11
1    11
3     7
5     3
Name: score, dtype: int64

In [4]:
ans.loc[:, "score"].value_counts()

2    39
4    33
3    31
1    30
5    13
Name: score, dtype: int64

In [5]:
my_ans = ans[ans["method"] == "psi_hwm"]
print(my_ans.describe())
print(my_ans.loc[:, "score"].value_counts())

           score
count  43.000000
mean    2.581395
std     1.366827
min     1.000000
25%     1.000000
50%     2.000000
75%     4.000000
max     5.000000
1    12
2    11
3     8
4     7
5     5
Name: score, dtype: int64


In [6]:
vik_ans = ans[ans["method"] != "psi_hwm"]
print(vik_ans.describe())
print(vik_ans.loc[:, "score"].value_counts())

            score
count  103.000000
mean     2.786408
std      1.225949
min      1.000000
25%      2.000000
50%      3.000000
75%      4.000000
max      5.000000
2    28
4    26
3    23
1    18
5     8
Name: score, dtype: int64


In [7]:
my_cls_desc = pd.read_csv("../data/my_classes.csv", delimiter=',',
                                quoting=csv.QUOTE_MINIMAL, error_bad_lines=True,
                                header=None, names=["class_name", "description"])
my_cls_list = ["big_code_hierarchy", "big_methods", "big_static_arrays_or_map", "complex_annotation", "kotlin_tutorials", "many_assertions", "many_case_in_when", "many_class_references", "many_delegate_properties", "many_empty_string_literals", "many_generic_parameters", "many_if_statements", "many_lambdas", "many_literal_strings", "many_local_variables", "many_loops", "many_nested_functions", "many_similar_call_expressions", "many_similar_code_fragments", "many_throws", "many_try_catch", "many_type_casts", "strange_code_constructs"]
# my_cls_list = my_cls_desc.loc[:, "class_name"]
my_cls = cls[cls["class_name"].isin(my_cls_list)]
print(my_cls.describe())

           score
count  23.000000
mean    2.565217
std     1.440520
min     1.000000
25%     1.000000
50%     2.000000
75%     4.000000
max     5.000000


In [8]:
vik_cls_list = list(set(vik_ans.loc[:, "class_name"]))
vik_cls = cls[cls["class_name"].isin(vik_cls_list)]
print(vik_cls.describe())

           score
count  30.000000
mean    2.933333
std     1.284747
min     1.000000
25%     2.000000
50%     3.000000
75%     4.000000
max     5.000000


In [9]:
print(my_cls.loc[:, "score"].value_counts())
# sns.distplot(my_cls.loc[:, "score"])

1    7
2    6
4    4
5    3
3    3
Name: score, dtype: int64


In [ ]:
X = np.array(my_cls.loc[:, "score"])
print(X)

plt.hist(X, range=(1, 5), bins=5)
plt.title("Оценки классов")
plt.xlabel("Оценка")
plt.ylabel("Кол-во классов")
plt.show()
# fig = plt.gcf()

In [34]:
my_cls.shape[0]

23

In [ ]:
useful_classes = list(my_cls[my_cls["score"] >= 4].loc[:, "class_name"])
print(useful_classes)
# useful_anomalies = my_ans[my_ans["class_name"].isin(useful_classes)]
# useful_anomalies.shape

In [11]:
# Table: my_classes, N_both, scores

my_ans = ans[ans["class_name"].isin(my_cls_list)]
ex_count = my_ans.groupby(["class_name"]).agg(['count'])
# ex_count_both = ex_count[ex_count.iloc[0].isin(my_cls_list)]
ex_count_both = np.array(ex_count.iloc[:, 1])[:,None]
print(ex_count.shape)

# ex_count_my = my_ans.groupby(["class_name"]).agg(['count'])
# ex_count_my = np.array(ex_count_my.iloc[:, 1])[:,None]
# print(ex_count_my.shape)

my_scores = np.array(my_cls.loc[:, "score"])[:,None]
print(my_scores.shape)

labels = np.array(my_cls_list)[:,None]
print(labels.shape)

table = np.column_stack((labels, ex_count_both, my_scores))
table = table[table[:,2].argsort()[::-1]]
table

(23, 3)
(23, 1)
(23, 1)


array([['many_generic_parameters', '3', '5'],
       ['many_case_in_when', '8', '5'],
       ['many_delegate_properties', '3', '5'],
       ['strange_code_constructs', '3', '4'],
       ['complex_annotation', '2', '4'],
       ['many_if_statements', '9', '4'],
       ['many_similar_call_expressions', '10', '4'],
       ['big_code_hierarchy', '8', '3'],
       ['big_methods', '5', '3'],
       ['many_try_catch', '1', '3'],
       ['many_loops', '4', '2'],
       ['big_static_arrays_or_map', '10', '2'],
       ['many_throws', '2', '2'],
       ['many_class_references', '2', '2'],
       ['many_similar_code_fragments', '2', '2'],
       ['many_lambdas', '2', '2'],
       ['many_empty_string_literals', '2', '1'],
       ['many_type_casts', '2', '1'],
       ['many_assertions', '1', '1'],
       ['kotlin_tutorials', '2', '1'],
       ['many_literal_strings', '4', '1'],
       ['many_local_variables', '2', '1'],
       ['many_nested_functions', '2', '1']],
      dtype='<U29')

In [11]:
# Table: my_class_descriptions, my_scores

indices = np.array([x for x in range(1, 24)])[:,None]

# labels = np.array(my_cls_desc.loc[:, "class_name"])[:,None]
desc = np.array(my_cls_desc.loc[:, "description"])[:,None]

ex_count = my_ans.groupby(["class_name"]).agg(['count'])
ex_count_both = np.array(ex_count.iloc[:, 1])[:,None]

my_scores = np.array(my_cls.loc[:, "score"])[:,None]

table = np.column_stack((desc, my_scores))
table = table[table[:,-1].argsort()[::-1]]
# table = np.hstack((indices, table))
print(table.shape)

# print(table[10])
print(table[11])
# print(table[12])
table = np.hstack((table[:11], table[12:]))

table = pd.DataFrame(table)
table.to_csv("../data/table_sorted.csv", header=False, index=False)
table

(23, 2)
['Большие литеральные массивы или словари' 2]


,0,1,2,3
0,Много типовых параметров,5,Много throw-операторов,2
1,Много веток в when-выражении,5,Много ссылок на класс (н-р,2
2,Много делегированных свойств,5,Много схожих фрагментов кода,2
3,Необычные кодовые конструкции,4,Много лямбда-выражений,2
4,Сложные аннотации функции,4,Много пустых строковых литералов,1
5,Много if-выражений,4,Много преобразований типов,1
6,Много схожих выражений,4,Много assert-операторов,1
7,Сложная структура кода,3,Код из руководств по Kotlin,1
8,Большое тело функции,3,Много строковых литералов,1
9,Много конструкций try-catch,3,Много локальных переменных,1


In [ ]:

# my_scores

In [12]:
# table.shape
df = pd.DataFrame(table)
df.to_csv("../data/table.csv", header=False, index=False)
df

,0,1,2
0,many_generic_parameters,3,5
1,many_case_in_when,8,5
2,many_delegate_properties,3,5
3,strange_code_constructs,3,4
4,complex_annotation,2,4
5,many_if_statements,9,4
6,many_similar_call_expressions,10,4
7,big_code_hierarchy,8,3
8,big_methods,5,3
9,many_try_catch,1,3


In [16]:
# Table: all classes, W_1, W_2, Q

indices = np.array([x for x in range(1, 44)])[:,None]
all_cls = np.array(cls.loc[:, "class_name"])[:,None]
plus_my = np.array(["+" if x in my_cls_list else "" for x in all_cls])[:,None]
plus_vik = np.array(["+" if x in vik_cls_list else "" for x in all_cls])[:,None]
all_marks = np.array(cls.loc[:, "score"])[:,None]

table_all = np.column_stack((all_cls, plus_my, plus_vik, all_marks))
table_all = table_all[table_all[:,3].argsort()[::-1]]

print(table_all[21])
table_all = np.hstack((table_all[:21], table_all[22:]))

table_all = pd.DataFrame(table_all)
table_all.to_csv("../data/table_all.csv", header=False, index=False)
table_all

['many_not_null_assertion_operators' '' '+' 2]


,0,1,2,3,4,5,6,7
0,many_case_in_when,+,+,5,many_similar_code_fragments,+,,2
1,many_delegate_properties,+,+,5,many_inline_functions,,+,2
2,many_generic_parameters,+,+,5,many_concatenations,,+,2
3,nested_calls,,+,4,many_class_references,+,,2
4,many_if_statements,+,+,4,many_lambdas,+,,2
5,big_constants_set,,+,4,many_throws,+,,2
6,complex_annotation,+,,4,many_type_reified_params,,+,2
7,long_calls_chain,,+,4,complex_or_long_logical_expressions,,+,2
8,long_enumerations,,+,4,many_loops,+,+,2
9,strange_code_constructs,+,,4,big_static_arrays_or_map,+,+,2


In [ ]:
my_ans